# Basalt DatasetSDK Async Demo

This notebook demonstrates the asynchronous functionality of the DatasetSDK in the Basalt Python SDK.

In [ ]:
import asyncio
from basalt import Basalt
import os

## Setup

Initialize the SDK with your API key. If you have the BASALT_API_KEY environment variable set, it will use that; otherwise, replace "your-api-key" with your actual API key.

In [ ]:
# Initialize the SDK with your API key
api_key = os.environ.get("BASALT_API_KEY", "your-api-key")

# Create a Basalt client
basalt = Basalt(api_key)

## Example 1: Asynchronously List All Datasets

This example demonstrates how to list all datasets asynchronously.

In [ ]:
async def list_datasets():
    print("Listing all datasets asynchronously...")
    err, datasets = await basalt.datasets.async_list()
    if err:
        print(f"Error listing datasets: {err}")
    else:
        print(f"Found {len(datasets)} datasets")
        for dataset in datasets:
            print(f"- {dataset.name} (slug: {dataset.slug})")
    return datasets

# Run the async function
datasets = await list_datasets()

## Example 2: Asynchronously Get a Specific Dataset

This example demonstrates how to retrieve a specific dataset by its slug.

In [ ]:
async def get_dataset(datasets):
    print("\nGetting a specific dataset asynchronously...")
    if len(datasets) > 0:
        sample_dataset = datasets[0]
        err, dataset = await basalt.datasets.async_get(sample_dataset.slug)
        if err:
            print(f"Error getting dataset: {err}")
        else:
            print(f"Retrieved dataset: {dataset.name}")
            print(f"Columns: {dataset.columns}")
            print(f"Number of rows: {len(dataset.rows) if dataset.rows else 0}")
        return sample_dataset, dataset
    else:
        print("No datasets available")
        return None, None

# Run the async function
sample_dataset, dataset = await get_dataset(datasets)

## Example 3: Asynchronously Add a Row to a Dataset

This example demonstrates how to add a new row to an existing dataset.

In [ ]:
async def add_row(sample_dataset):
    print("\nAdding a row to a dataset asynchronously...")
    if sample_dataset:
        # Create some sample values for the dataset row
        values = {column: f"Sample {column} value" for column in sample_dataset.columns}
        
        err, row, warning = await basalt.datasets.async_addRow(
            slug=sample_dataset.slug,
            values=values,
            name="Async Sample Row",
            ideal_output="Expected output for this row",
            metadata={"source": "async_example", "type": "demo"}
        )
        
        if err:
            print(f"Error adding row to dataset: {err}")
        elif warning:
            print(f"Row added with warning: {warning}")
            print(f"Row values: {row.values}")
        else:
            print(f"Row added successfully")
            print(f"Row values: {row.values}")
            print(f"Row name: {row.name}")

# Run the async function
await add_row(sample_dataset)

## Example 4: Asynchronously Get a Dataset as an Object

This example demonstrates how to retrieve a dataset as an object for further manipulation.

In [ ]:
async def get_dataset_object(sample_dataset):
    print("\nGetting a dataset as an object asynchronously...")
    if sample_dataset:
        dataset_obj = await basalt.datasets.async_get_dataset_object(sample_dataset.slug)
        if dataset_obj:
            print(f"Retrieved dataset object: {dataset_obj.name}")
            print(f"Number of rows: {len(dataset_obj.rows)}")
        else:
            print("Failed to get dataset object")
        return dataset_obj
    else:
        print("No sample dataset available")
        return None

# Run the async function
dataset_obj = await get_dataset_object(sample_dataset)

## Example 5: Asynchronously Add a Row to a Dataset Object

This example demonstrates how to add a row directly to a dataset object.

In [ ]:
async def add_row_to_dataset_object(dataset_obj):
    print("\nAdding a row to a dataset object asynchronously...")
    if dataset_obj:
        # Create some sample values for the dataset row
        values = {column: f"Another sample {column} value" for column in dataset_obj.columns}
        
        row = await basalt.datasets.async_add_row_to_dataset(
            dataset=dataset_obj,
            values=values,
            name="Another Async Sample Row",
            ideal_output="Another expected output",
            metadata={"source": "async_dataset_object_example", "type": "demo"}
        )
        
        if row:
            print(f"Row added to dataset object successfully")
            print(f"Row values: {row.values}")
            print(f"Updated number of rows in dataset object: {len(dataset_obj.rows)}")
        else:
            print("Failed to add row to dataset object")

# Run the async function
await add_row_to_dataset_object(dataset_obj)

## Example 6: Execute Multiple Dataset Operations Concurrently

This example demonstrates how to execute multiple asynchronous operations concurrently for better performance.

In [ ]:
async def execute_concurrent_operations(sample_dataset):
    print("\nExecuting multiple dataset operations concurrently...")
    
    # Create multiple async tasks
    tasks = [
        basalt.datasets.async_list(),
        basalt.datasets.async_get(sample_dataset.slug) if sample_dataset else None
    ]
    
    # Filter out None tasks
    tasks = [t for t in tasks if t is not None]
    
    if tasks:
        # Execute all tasks concurrently
        results = await asyncio.gather(*tasks)
        
        print(f"Completed {len(tasks)} operations concurrently")
        print(f"Number of datasets listed: {len(results[0][1]) if results[0][1] else 'Error'}")
        if len(tasks) > 1:
            print(f"Retrieved dataset: {results[1][1].name if results[1][1] else 'Error'}")

# Run the async function
await execute_concurrent_operations(sample_dataset)